In [10]:
import numpy as np  
import pickle
from sklearn.decomposition import PCA
import gpflow

In [11]:
def modelLoad(nRankMax, snap_ID):
    GPmodel = modelDir + 'GP_smooth_rank' + str(nRankMax) + 'snap' + str(snap_ID)  
    PCAmodel = modelDir + 'PCA_smooth_rank' + str(nRankMax) + 'snap' + str(snap_ID)  

    ctx_for_loading = gpflow.saver.SaverContext(autocompile=False)
    saver = gpflow.saver.Saver()
    GPm = saver.load(GPmodel, context=ctx_for_loading)
    GPm.clear()
    GPm.compile()
    PCAm = pickle.load(open(PCAmodel, 'rb'))
    return GPm, PCAm

def GPy_predict(gpmodel, para_array):
    m1p = gpmodel.predict_f(para_array)  # [0] is the mean and [1] the predictive
    W_predArray = m1p[0]
    W_varArray = m1p[1]
    return W_predArray, W_varArray


def Emu(gpmodel, pcamodel, para_array):
    para_array = np.array(para_array)
    if len(para_array.shape) == 1:
        W_predArray, _ = GPy_predict(gpmodel, np.expand_dims(para_array, axis=0))
        x_decoded = pcamodel.inverse_transform(W_predArray)
        return np.squeeze(x_decoded)#[0]

def MGemu(Om, ns, s8, fR0, n, z):
    snap_idx_nearest = (np.abs(z_all - z)).argmin()
    if (z > z_all[snap_idx_nearest]): 
        snap_ID_z1 = snap_idx_nearest - 1    
    else:
        snap_ID_z1 = snap_idx_nearest 
    snap_ID_z2 = snap_ID_z1 + 1

    GPm1, PCAm1 = modelLoad(nRankMax, snap_ID_z1)
    Pk_z1 = Emu(GPm1, PCAm1, [Om, ns, s8, fR0, n])
    z1 = z_all[snap_ID_z1]

    GPm2, PCAm2 = modelLoad(nRankMax, snap_ID_z2)
    Pk_z2 = Emu(GPm2, PCAm2, [Om, ns, s8, fR0, n])
    z2 = z_all[snap_ID_z2]
    
    # print('Linear interpolating for redshift between', 'z1= '+ str(z_all[snap_ID_z1]), 'z2= '+ str(z_all[snap_ID_z2]))
    Pk_interp = np.zeros_like(Pk_z1)
    Pk_interp = Pk_z2 + (Pk_z1 - Pk_z2)*(z - z2)/(z1 - z2)
    return Pk_interp


In [12]:
nRankMax = 16
snap_ID = 61
del_idx = [5, 25, 4, 42]  ## Random holdouts (not used in training, reserved for validation) 


dataDir = "./Data/Emulator213bins/" ## Data folder
modelDir = "./Models/" ## Data folder

az = np.loadtxt(dataDir + 'timestepsCOLA.txt', skiprows=1) 
fileIn = dataDir + 'ratiosbins_' + str(99) + '.txt'

loadFile = np.loadtxt(fileIn)
PmPl_all = loadFile[:, 1:].T
kvals = loadFile[:,0]
z_all = az[:, 1]

In [14]:
Om, ns, s8, fR0, n = 0.1, 1.0, 0.8, 3e-5, 1.5

MGemu(Om=0.1, ns=1.0, s8=0.8, fR0=3e-5, n=1.5, z=0.02)

/Users/nramachandra/anaconda3/envs/env_py37/lib/python3.7/site-packages/gpflow/saver/serializers.py:46: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(pathname_or_file_like) as h5file:
/Users/nramachandra/anaconda3/envs/env_py37/lib/python3.7/site-packages/gpflow/saver/serializers.py:47: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return h5file['data'].value


array([1.00765992, 1.0088824 , 1.01002134, 1.01108021, 1.01206245,
       1.01297153, 1.01381092, 1.01458407, 1.01529445, 1.0159455 ,
       1.0165407 , 1.01708351, 1.01757737, 1.01802576, 1.01843214,
       1.01879996, 1.01913268, 1.01943377, 1.01970668, 1.01995488,
       1.02018183, 1.02039098, 1.02058579, 1.02076973, 1.02094626,
       1.02111884, 1.02129125, 1.02150745, 1.02190258, 1.0220049 ,
       1.02213614, 1.02265642, 1.02318918, 1.02374871, 1.02414526,
       1.0245813 , 1.02509907, 1.02547093, 1.02556506, 1.02597648,
       1.02647479, 1.026804  , 1.02729316, 1.02764825, 1.02811849,
       1.02850589, 1.02907893, 1.02969218, 1.03000498, 1.03045773,
       1.03088826, 1.03137245, 1.03159913, 1.03192888, 1.0326679 ,
       1.03297599, 1.03383508, 1.03471794, 1.03555367, 1.03604795,
       1.03660514, 1.03718702, 1.03782189, 1.03836419, 1.03910928,
       1.03965976, 1.04029974, 1.04107061, 1.04189555, 1.04248605,
       1.04316053, 1.04393349, 1.04459205, 1.04528369, 1.04617